In [6]:
import os
import torch
from torch import nn
import pytorch_lightning as pl
import numpy as np

In [26]:
class MLP(pl.LightningModule):

    def __init__(self, input_dim, output_dim, lr):
        super().__init__()
        
        self.learning_rate = lr
        
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, output_dim)
        )
        
        self.mean_error = nn.MSELoss()

    def forward(self, x):
        
        return self.layers(x)

    def training_step(self, batch, batch_idx):
        print(len(batch))
        x, y = batch
        
        prediction = self.layers(x)
        loss = self.mean_error(prediction, y)
        self.log('training_loss', loss)
        
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        
        return optimizer

In [27]:
dummy_test_data = np.array([
    [4.,7., 800.],
    [4.,8., 900.],
    [1.,4., 600.],
    [2.,3., 200.]
])
dummy_train_data = np.array([
    [5.,8., 1000.],
    [6.,9., 1050.],
])

In [37]:

print(train_x.size())
print(train_y)
test_x = torch.tensor(dummy_test_data[:,0:2], dtype=torch.double)
print(test_x)

torch.Size([2, 2])
tensor([1000., 1050.], dtype=torch.float64)
tensor([[4., 7.],
        [4., 8.],
        [1., 4.],
        [2., 3.]], dtype=torch.float64)


In [36]:
# IterableDataset
class CustomDataset(torch.utils.data.IterableDataset):
    def __init__(self, data):
        train_x = torch.tensor(data[:,0:2], dtype=torch.double)
        train_y = torch.tensor((data[:, 2:]).flatten(), dtype=torch.double)
        zip_data = tuple(zip(train_x, train_y))
        self.data_source = zip_data

    def __iter__(self):
        print('ok')
        return iter(self.data_source)

In [39]:
input_dim = 2
output_dim = 1
lr= 0.004

model = MLP(input_dim, output_dim, lr)
model = model.float()
trainer = pl.Trainer()
train_dataloader_test = CustomDataset(dummy_train_data)
trainer.fit(model, train_dataloader_test)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/harvey/spaceapps-2022/lightning_logs

  | Name       | Type       | Params
------------------------------------------
0 | layers     | Sequential | 2.3 K 
1 | mean_error | MSELoss    | 0     
------------------------------------------
2.3 K     Trainable params
0         Non-trainable params
2.3 K     Total params
0.009     Total estimated model params size (MB)


Epoch 0: : 0it [00:00, ?it/s]ok
2


RuntimeError: expected scalar type Double but found Float